# 실시간 프로그램 클론코딩

In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QAxContainer import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import datetime

In [ ]:
class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setGeometry(300, 300, 400, 300)
        self.setWindowTitle("실시간 프로그램")
        
        self.range = None
        self.target = None
        self.account = None
        self.amount = None
        self.hold = None
        
        self.previous_day_hold = False
        self.previous_day_quantity = False
        
        self.plain_text_edit = QPlainTextEdit(self)
        self.plain_text_edit.setReadOnly(True)
        self.plain_text_edit.move(10, 10)
        self.plain_text_edit.resize(380, 280)
        
        self.ocx = QAxWidget("KHOPENAPI.KHOpenAPICtrl.1")
        self.ocx.OnEventConnect.connect(self._handler_login)
        self.ocx.OnReceiveTrData.connect(self._handler_tr_data)
        self.ocx.OnReceiveRealData.connect(self._handler_real_data)
        self.ocx.OnReceiveChejanData.connect(self._handler_chejan_data)
        
        self.login_event_loop = QEventLoop()
        self.CommConnect()
        self.run()
        
    def CommConnect(self):
        self.ocx.dynamicCall("CommConnect()")
        self.login_event_loop.exec()
        
    def run(self):
        accounts = self.GetLoginInfo("ACCNO")
        self.account = accounts.split(";")[0]
        print(self.account)
        
        # TR 요청
        self.request_opt10081()
        self.request_opw00001()
        self.request_opw00004()
        
        # 실시간 주식체결
        self.subscribe_market_time("1")
        self.subscribe_stock_conclusion("2")
        
    def GetLoginInfo(self, tag):
        data = self.ocx.dynamicCall("GetLoginInfo(QString)", tag)
        return data
        
        
        
        